# Compare models experiments with PointX

In [ ]:
import pandas as pd
import os, json, sqlite3, asyncio, random, re
from dotenv import load_dotenv
from openai import OpenAI

from SchemaLinking import SchemaLinking
import warnings
from transformers import AutoTokenizer, AutoModelForCausalLM
import google.generativeai as genai

In [ ]:
load_dotenv('../.env')
tokenizer = AutoTokenizer.from_pretrained("../models/nsql-350M")
model = AutoModelForCausalLM.from_pretrained("../models/nsql-350M")

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
DEEPSEEK_API_KEY = os.environ.get('DEEPSEEK_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
system_content_puresql = """You are a helpful assistant for generate SQL query from user-specified questions. 
please return only answer of sql string query result !!! 
Do not return any other format the user has provided to you.
This is example of output format which user expect from you
query : 'SELECT...'
"""

system_content_schemaprovide = """You are a helpful assistant for generate SQL query from user-specified questions and schema.
User provides you with a question.
Please return only a sql string query results.
Do not return any other format the user has provided to you.
This is example of output format which user expect from you
query : 'SELECT...'
"""

system_content_fillmask = """You are a helpful assistant for generate SQL query from user-specified questions and schema. 
User has some SQL where the [MASK] columns, condition values and tables are syntaxed and User wants you to respond to output that populates the [MASK] column of the SQL input followed by the question and schema description (name - description - data type).
If you don't know which column to fill in Do not include columns that you have created yourself. And only columns defined from the schema must be used. 
Do not use columns from other tables or schema. must also be used from the same table defined in the input.
If you must enter conditional values Please decide the format or value based on the sample values of that column.
If that column has many too long category value please decide base on column description.
please return only the answer of sql string query result!!! ('SELECT...')
"""

In [ ]:
zero_shot_prompt_mask = """For example:
table :     cat - this table contain cat information 
columns :    id - number for identify cat | number
            name - name of cat | text
            age - age of cat | number
            birth_date - pet birthday in format 'YYYY-MM-DD' | datetime
            gender - gender of cat (male, female) | text

question : Show me number of cat for each gender which born before March 23, 2011.
input : SELECT [MASK], COUNT([MASK]) FROM [MASK] WHERE [MASK] < [MASK] GROUP BY [MASK] ;
query : SELECT gender, COUNT(*) FROM cat WHERE birth_date < '2011-03-23' GROUP BY gender;

"""

zero_shot_prompt = """For example:
table :     cat - this table contain cat information 
columns :    id - number for identify cat | number
            name - name of cat | text
            age - age of cat | number
            birth_date - pet birthday in format 'YYYY-MM-DD' | datetime
            gender - gender of cat (male, female) | text

question : 'Show me number of cat for each gender which born before March 23, 2011.'
query : 'SELECT gender, COUNT(*) FROM cat WHERE birth_date < '2011-03-23' GROUP BY gender;'

"""

In [ ]:
def create_nsql_prompt(schema_link:object, question:str, used_schema:dict) -> str:
    """
    Generate a prompt for applying into SQL generation model based on the question and schema.

    Parameters:
    schema_link (object): The instance of the class containing schema information.
    question (str): The question for which the prompt is generated.
    used_schema (dict): A dictionary containing tables as keys and lists of columns as values after filtering the schema.

    Returns:
    str: A prompt for applying into SQL generation model.

    Example:
    prompt = create_prompt(schema_instance, "What are the total sales?", 
                          { 'sales': {'date' : 0.3, 'amount' : 0.61}, 
                            'products': {'name' : 0.23, 'price' : 0.57}})
    print(prompt)

    CREATE TABLE sales ( date DATE, amount INT,PRIMARY KEY ("date") )
    -- Using valid SQLite, answer the following questions for the tables provided above.
    -- What are the total sales?
    SELECT
    """
    full_sql = ""
    for table, columns in used_schema.items():
        if not len(columns): continue       # pass this table when no column
        primary_keys = schema_link.schema_datatypes[table]["JOIN_KEY"]["PK"]
        foreign_keys = list(schema_link.schema_datatypes[table]["JOIN_KEY"]["FK"].keys())
        join_table_key = primary_keys + foreign_keys
        
        sql = f"CREATE TABLE {table} ("
        for column in columns:
            if column in join_table_key and len(join_table_key): join_table_key.remove(column)
            try:
                sql += f' {column} {schema_link.schema_datatypes[table]["COLUMNS"][column]},'
            except KeyError: 
                print(f"KeyError :{column}")
                
        if len(join_table_key): # key for join of table are remaining
            for column in join_table_key:
                sql += f' {column} {schema_link.schema_datatypes[table]["COLUMNS"][column]},'

        # All table contain PK (maybe)
        if len(primary_keys):
            sql += 'PRIMARY KEY ('
            for pk_type in primary_keys: sql += f'"{pk_type}" ,'
            sql = sql[:-1] + "),"

        if len(foreign_keys):
            for fk, ref_table_column in schema_link.schema_datatypes[table]["JOIN_KEY"]["FK"].items():
                sql += f' FOREIGN KEY ("{fk}") REFERENCES "{list(ref_table_column.keys())[0]}" ("{list(ref_table_column.values())[0]}"),'

        sql = sql[:-1] + " )\n\n"
        full_sql += sql
    prompt = full_sql + "-- Using valid SQLite, answer the following questions for the tables provided above."
    prompt = prompt + '\n' + '-- ' + question
    prompt = prompt + '\n' + "SELECT"

    return prompt

def generate_nsql_sql(prompt):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        generated_ids = model.generate(input_ids, max_length=1000)
        sql = tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('\n')[-1]
    return sql

async def LLM_gensql(full_prompt:str, system_content:str, llm_model:str) -> str:
        """
        Generate SQL query followed by prompt

        Parameters:
        prompt (str): prompt for generate result
        llm_model (str): model-service name for generate result

        Returns:
        str: The complete SQL query.
        """
        if llm_model in ['gemini-pro']:
            try:
                gemini_prompt = system_content + full_prompt
                genai.configure(api_key=GOOGLE_API_KEY)
                gemini_model = genai.GenerativeModel(llm_model)
                gemini_model.temperature = 0
                response = gemini_model.generate_content(gemini_prompt)
                return response.text
            
            except Exception as e:
                 return f"Google AI Error : {e}"
            
        elif llm_model in ['gpt-3.5-turbo', 'gpt-4-0125-preview']:
            
            API_KEY = OPENAI_API_KEY
            base_url = "https://api.openai.com/v1"
            # return None
        
        elif llm_model in ['deepseek-coder', 'deepseek-chat']:
            API_KEY = DEEPSEEK_API_KEY
            base_url = "https://api.deepseek.com/v1"
        try:
            print(llm_model)
            
            client = OpenAI(api_key=API_KEY, base_url=base_url)
            response = client.chat.completions.create(
                model=llm_model,
                messages=[
                        {"role": "system",
                            "content": system_content},
                        {"role": "user", 
                            "content": full_prompt},
                        ],
                stop=['\n'],
                temperature=0
            )
            return response.choices[0].message.content
        
        except Exception as e:
            return f"API Error :{e}"
            
def get_reason(schema_link:object, sql_result:str) -> str:
    """
    Get the reason message related to the selected columns and tables from the schema based on the SQL query.

    Parameters:
    schema_link (object): The instance of the class containing schema information.
    sql_result (str): The SQL query result for which the reason message is generated.

    Returns:
    str: The reason message explaining the selection of columns and tables from the schema.

    Example:
    get_reason(schema_instance, "SELECT column1, column2 FROM table1 WHERE column3 = 'value'")

    Table - table1 : Description of table1
        Column - column1 : Description of column1
        Column - column2 : Description of column2
        Column - column3 : Description of column3
    """

    table_col_sql = schema_link.table_col_of_sql(sql_result)
    reason = ""

    for table, cols in table_col_sql.items():
        _df = schema_link.column_info_df[schema_link.column_info_df['Table'] == table][['Column', 'Description']].drop_duplicates()
        table_reason = f"Table - {table}\t: {schema_link.table_descriptions[table]['text']}\n"
        if len(cols):       # have columns of table
            col_reason = "\n".join([f"\tColumn - {c}\t: {_df.loc[_df['Column'] == c, 'Description'].values[0]}" for c in cols])
        else: col_reason = ""
        reason += str(table_reason + col_reason + "\n\n")

    return reason

In [ ]:
def create_llm_prompt(schema_link:object, used_schema:dict, question:str, masked_query:str, 
                      few_shot:str=zero_shot_prompt_mask, is_marked:bool=True, is_fewshot:bool=True) -> str:

    full_prompt = ""
    for table_name, column_score in used_schema.items():
        _df = schema_link.column_info_df[schema_link.column_info_df['Table'] == table_name][['Column', 'Description']].drop_duplicates()
        full_prompt += f"\ntable : {table_name} - {schema_link.table_descriptions[table_name]['text']}\ncolumns:"

        for column_name in column_score:
            full_prompt += f"\t{column_name} - {_df[_df['Column'] == column_name]['Description'].values[0]}"
            full_prompt += f" | {schema_link.schema_datatypes[table_name]['COLUMNS'][column_name]}\n"

    full_prompt += f"question : {question}\n"
    if is_marked: full_prompt += f"input : {masked_query}"
    if is_fewshot: full_prompt = few_shot + full_prompt
    
    return full_prompt + "\nquery : "

In [ ]:
def query_pointx_db(sql_query):
    try:
        conn = sqlite3.connect(f'../src/pointx/database/pointx.db')
        cursor = conn.cursor()
    except:
        return "CANNOT CONNECT DATABASE"
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
    except:
        return "CANNOT FETCHING DATA"
    conn.close()
    return results

In [ ]:
def table_col_of_sql(schema_link, sql_query:str) -> dict:
        """
        Extract tables and their corresponding columns from the given SQL query.

        Parameters:
        sql_query (str): The SQL query from which tables and columns need to be extracted.

        Returns:
        dict: A dictionary containing tables as keys and lists of columns as values.

        Example:
        SchemaLinking.table_col_of_sql("SELECT column1, column2 FROM table1 WHERE column3 = 'value'")
        {'table1': ['column1', 'column2', 'column3']}
        """
        
        selected_schema = {}
        query_split = re.split(schema_link.split_pattern, sql_query)
        for table in schema_link.schema_datatypes.keys():
            if table in query_split:
                selected_col = []
                for col in schema_link.schema_datatypes[table]['COLUMNS'].keys():
                    if col in query_split: selected_col.append(col)
                selected_schema[table] = selected_col

        return selected_schema

In [ ]:
with open(f"../src/src_dev/pointx/embedded_data.json", "r") as f:
    domain = json.load(f)

schema_link = SchemaLinking(domain)

async def ChatQ_pipeline(question:str, domain_tables:list, llm_model_name:str, 
                         max_n:int=10,verbose:bool=True, get_final_prompt:bool=False):

    if not domain_tables: domain_tables = list(domain['tables'].keys())
    used_schema = schema_link.filter_schema(question, domain_tables, max_n=max_n)
    nsql_prompt = create_nsql_prompt(schema_link, question, used_schema)
    nsql_sql_result = generate_nsql_sql(nsql_prompt)
    masked_query = schema_link.masking_query(nsql_sql_result)
    llm_prompt = create_llm_prompt(schema_link, used_schema, question, masked_query)
    if get_final_prompt: return llm_prompt
    llm_result = await LLM_gensql(llm_prompt, system_content_fillmask, llm_model_name)
    if verbose:
        reason = get_reason(schema_link, llm_result)
        print("========= QUESTION =========")
        print(question)
        print()
        print("========= NSQL SQL =========")
        print(nsql_sql_result)
        print()
        print("========= LLM SQL =========")
        print(llm_result)
        print()
        print("========= REASON =========")
        print(reason)
        print()
        print("========= SCHEMA =========")
        print(used_schema)
        print()

    return llm_result

In [ ]:
q_pair_df = pd.read_csv("../src/pointx/PointX_questionpair.csv")[['Table', 'Question', 'Actual SQL']]
print(q_pair_df.shape)
q_pair_df.head()

In [ ]:
# # delete duplicate experiment record

# _df = pd.read_excel('results/temp_model_description_experiments.xlsx')
# print(_df.shape)
# _df.drop_duplicates(subset=['Question'], inplace=True)
# print(_df.shape)
# _df.to_excel('results/temp_model_description_experiments.xlsx', index=False)

In [ ]:
query = await ChatQ_pipeline("How many users have not used the app for more than 3 months?",
                            ['pointx_fbs_rpt_dly'], "gpt-3.5-turbo", verbose=True)
query

## GenAI Model with provide schema-description

In [ ]:
def split_list(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]

In [ ]:
def sample_columns(schema_link:object, table_name:str, used_columns:dict, noise:int=20) -> dict:
    table_columns = list(schema_link.column_info_df[schema_link.column_info_df['Table'] == table_name]['Column'].unique())
    remaining_values = [value for value in table_columns if value not in used_columns[table_name]]
    remaining_count = noise - len(used_columns[table_name])
    random_selected_values = random.sample(remaining_values, remaining_count)
    result = list(used_columns[table_name]) + random_selected_values
    used_schema = { table_name : result}
    return used_schema

In [ ]:
def yeild_columns(schema_link, sql_queries:list) -> list:

    selected_schema = {}
    for sql_query in sql_queries:
        
        query_split = re.split(schema_link.split_pattern, sql_query)
        
        for table in schema_link.schema_datatypes.keys():
            if table in query_split:
                selected_col = []
                for col in schema_link.schema_datatypes[table]['COLUMNS'].keys():
                    if col in query_split:
                        selected_col.append(col)
                selected_schema[table] = selected_col
    
    return selected_schema


In [ ]:
sample_columns(schema_link, 'pointx_fbs_rpt_dly', { "pointx_fbs_rpt_dly":{'month_id':0.2}}, 10)

In [ ]:
predict_data = {
    "Question" : [],
    "Desc DeepSeek" : [],
    "Desc GPT3.5" : [],
    "Desc GPT4" : [],
    "Desc Gemini" : []
}


i = 0
n_chunk = 5

temp_result_file = "results/temp_model_description_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()


for table_name in q_pair_df['Table'].unique():
    table_df = q_pair_df[q_pair_df['Table'] == table_name]
    table_questions = table_df['Question'].to_list()
    table_actualSQL = table_df['Actual SQL'].to_list()
    # list_chunk_questions = list(split_list(table_questions, n_chunk))
    # list_chunk_actualSQL = list(split_list(table_actualSQL, n_chunk))
    # cannot used full schema because context in larger than handle
    # used_schema = { table_name : list(schema_link.column_info_df[schema_link.column_info_df['Table'] == table_name]['Column'].unique())}
    
    
    for question, sql in zip(table_questions, table_actualSQL):
        if question in predict_data['Question']: continue
        # used_columns = yeild_columns(schema_link,sql)
        used_columns = table_col_of_sql(schema_link, sql)
        used_schema = sample_columns(schema_link, table_name, used_columns)
        table_prompt = create_llm_prompt(schema_link, used_schema, question, zero_shot_prompt, is_marked=False)

        gemini_results = LLM_gensql(table_prompt, system_content_schemaprovide, 'gemini-pro')
        gpt3_5_results = LLM_gensql(table_prompt, system_content_schemaprovide, 'gpt-3.5-turbo')
        gpt4_results = LLM_gensql(table_prompt, system_content_schemaprovide, 'gpt-4-0125-preview')
        deepseek_result = LLM_gensql(table_prompt, system_content_schemaprovide, 'deepseek-coder')

        print("Generating SQL...")
        gemini_results, deepseek_result, gpt3_5_results, gpt4_results = await asyncio.gather(gemini_results, deepseek_result, gpt3_5_results, gpt4_results)

        print(question)
        print(gemini_results, deepseek_result, gpt3_5_results, gpt4_results, sep='\n')

        predict_data['Question'].append(question)
        predict_data['Desc DeepSeek'].append(deepseek_result)
        predict_data['Desc GPT3.5'].append(gpt3_5_results)
        predict_data['Desc GPT4'].append(gpt4_results)
        predict_data['Desc Gemini'].append(gemini_results)

        i += 1
        if not i % 5:
            save_df = pd.DataFrame(predict_data)
            save_df.to_excel(temp_result_file, index=False)
            print("SAVE TEMP COMPLETE", i)
        
    save_df = pd.DataFrame(predict_data)
    save_df.to_excel(temp_result_file, index=False)
    print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_description_experiments.xlsx", index=False)

## GenAI Model with Framework pipeline

In [ ]:
predict_data = {
    "Question" : [],
    "Desc DeepSeek" : [],
    "Desc GPT3.5" : [],
    "Desc GPT4" : [],
    "Desc Gemini" : []
}


i = 0
n_chunk = 5

temp_result_file = "results/temp_model_chatq_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()


for table_name in q_pair_df['Table'].unique():
    table_df = q_pair_df[q_pair_df['Table'] == table_name]
    table_questions = table_df['Question'].to_list()
    table_actualSQL = table_df['Actual SQL'].to_list()
    
    
    for question, sql in zip(table_questions, table_actualSQL):
        if question in predict_data['Question']: continue
        
        table_prompt = await ChatQ_pipeline(question, domain_tables=[table_name], 
                                            llm_model_name=None, max_n=10,get_final_prompt=True)
        gemini_results = LLM_gensql(table_prompt, system_content_fillmask, 'gemini-pro')
        gpt3_5_results = LLM_gensql(table_prompt, system_content_fillmask, 'gpt-3.5-turbo')
        gpt4_results = LLM_gensql(table_prompt, system_content_fillmask, 'gpt-4-0125-preview')
        deepseek_result = LLM_gensql(table_prompt, system_content_fillmask, 'deepseek-coder')

        print("Generating SQL...")
        gemini_results, deepseek_result, gpt3_5_results, gpt4_results = await asyncio.gather(gemini_results, deepseek_result, gpt3_5_results, gpt4_results)

        print(question)
        print(gemini_results, deepseek_result, gpt3_5_results, gpt4_results, sep='\n')

        predict_data['Question'].append(question)
        predict_data['Desc DeepSeek'].append(deepseek_result)
        predict_data['Desc GPT3.5'].append(gpt3_5_results)
        predict_data['Desc GPT4'].append(gpt4_results)
        predict_data['Desc Gemini'].append(gemini_results)

        i += 1
        if not i % 5:
            save_df = pd.DataFrame(predict_data)
            save_df.to_excel(temp_result_file, index=False)
            print("SAVE TEMP COMPLETE", i)
        
    save_df = pd.DataFrame(predict_data)
    save_df.to_excel(temp_result_file, index=False)
    print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_chatq_experiments.xlsx", index=False)

## Pure question without providing schema and description

In [ ]:
predict_data = {
    "Question" : [],
    "Desc DeepSeek" : [],
    "Desc GPT3.5" : [],
    "Desc GPT4" : [],
    "Desc Gemini" : []
}


i = 0
n_chunk = 5

temp_result_file = "results/temp_model_pureQ_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()


for table_name in q_pair_df['Table'].unique():
    table_df = q_pair_df[q_pair_df['Table'] == table_name]
    table_questions = table_df['Question'].to_list()
    table_actualSQL = table_df['Actual SQL'].to_list()
    
    
    for question, sql in zip(table_questions, table_actualSQL):
        if question in predict_data['Question']: continue

        gemini_results = LLM_gensql(question, system_content_puresql, 'gemini-pro')
        gpt3_5_results = LLM_gensql(question, system_content_puresql, 'gpt-3.5-turbo')
        gpt4_results = LLM_gensql(question, system_content_puresql, 'gpt-4-0125-preview')
        deepseek_result = LLM_gensql(question, system_content_puresql, 'deepseek-coder')

        print("Generating SQL...")
        gemini_results, deepseek_result, gpt3_5_results, gpt4_results = await asyncio.gather(gemini_results, deepseek_result, gpt3_5_results, gpt4_results)

        print(question)
        print(gemini_results, deepseek_result, gpt3_5_results, gpt4_results, sep='\n')

        predict_data['Question'].append(question)
        predict_data['Desc DeepSeek'].append(deepseek_result)
        predict_data['Desc GPT3.5'].append(gpt3_5_results)
        predict_data['Desc GPT4'].append(gpt4_results)
        predict_data['Desc Gemini'].append(gemini_results)

        i += 1
        if not i % 5:
            save_df = pd.DataFrame(predict_data)
            save_df.to_excel(temp_result_file, index=False)
            print("SAVE TEMP COMPLETE", i)
        
    save_df = pd.DataFrame(predict_data)
    save_df.to_excel(temp_result_file, index=False)
    print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_pureQ_experiments.xlsx", index=False)

## Pure SQL

In [ ]:
predict_data = {
    "Question" : [],
    "ChatQ - NSQL" : []
}


i = 0
n_chunk = 5

temp_result_file = "results/temp_model_onlyNSQL_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()


for table_name in q_pair_df['Table'].unique():
    table_df = q_pair_df[q_pair_df['Table'] == table_name]
    table_questions = table_df['Question'].to_list()
    table_actualSQL = table_df['Actual SQL'].to_list()
    
    
    for question, sql in zip(table_questions, table_actualSQL):
        if question in predict_data['Question']: continue

        gemini_results = LLM_gensql(question, system_content_puresql, 'gemini-pro')
        gpt3_5_results = LLM_gensql(question, system_content_puresql, 'gpt-3.5-turbo')
        gpt4_results = LLM_gensql(question, system_content_puresql, 'gpt-4-0125-preview')
        deepseek_result = LLM_gensql(question, system_content_puresql, 'deepseek-coder')

        print("Generating SQL...")
        gemini_results, deepseek_result, gpt3_5_results, gpt4_results = await asyncio.gather(gemini_results, deepseek_result, gpt3_5_results, gpt4_results)

        print(question)
        print(gemini_results, deepseek_result, gpt3_5_results, gpt4_results, sep='\n')

        predict_data['Question'].append(question)
        predict_data['Desc DeepSeek'].append(deepseek_result)
        predict_data['Desc GPT3.5'].append(gpt3_5_results)
        predict_data['Desc GPT4'].append(gpt4_results)
        predict_data['Desc Gemini'].append(gemini_results)

        i += 1
        if not i % 5:
            save_df = pd.DataFrame(predict_data)
            save_df.to_excel(temp_result_file, index=False)
            print("SAVE TEMP COMPLETE", i)
        
    save_df = pd.DataFrame(predict_data)
    save_df.to_excel(temp_result_file, index=False)
    print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_pureQ_experiments.xlsx", index=False)

# NSQL only

In [ ]:
predict_data = {
    "Question" : [],
    "pure NSQL" : []
}


i = 0
n_chunk = 5

temp_result_file = "results/temp_model_pureNSQL_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()


for table_name in q_pair_df['Table'].unique():
    table_df = q_pair_df[q_pair_df['Table'] == table_name]
    table_questions = table_df['Question'].to_list()
    table_actualSQL = table_df['Actual SQL'].to_list()
    
    
    for question, sql in zip(table_questions, table_actualSQL):
        if question in predict_data['Question']: continue
        
        used_schema = schema_link.filter_schema(question, [table_name], max_n=50)
        nsql_prompt = create_nsql_prompt(schema_link, question, used_schema)
        nsql_sql_result = generate_nsql_sql(nsql_prompt)

        predict_data['Question'].append(question)
        predict_data['pure NSQL'].append(nsql_sql_result)

        i += 1
        if not i % 5:
            save_df = pd.DataFrame(predict_data)
            save_df.to_excel(temp_result_file, index=False)
            print("SAVE TEMP COMPLETE", i)
        
    save_df = pd.DataFrame(predict_data)
    save_df.to_excel(temp_result_file, index=False)
    print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_pureNSQL_experiments.xlsx", index=False)

# Structure by masking

In [ ]:
import pandas as pd
import json
from SchemaLinking import SchemaLinking

In [ ]:
import re

def extract_sql_query(text):
    
    sql_patterns = [r'```sql(.*?)```', r'```(.*?)```', r'(SELECT.*?;)', r'(SELECT.*)']
    
    for pattern in sql_patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return text


In [ ]:
with open(f"../src/src_dev/pointx/embedded_data.json", "r") as f:
    domain = json.load(f)

schema_link = SchemaLinking(domain)

result_file = pd.read_excel('../src/pointx/NLQ2SQL model exp result.xlsx').iloc[:,3:]
result_file = result_file.applymap(str)
for col in result_file.columns:
    result_file[col] = result_file[col].apply(extract_sql_query)
result_file.head()

In [ ]:
ncols = len(result_file.columns) // 2
sql_df = result_file.iloc[:,:ncols]
sql_df.head()

In [ ]:
schema_link.masking_query("SELECT customer_type, SUM(ntx) FROM pointx_cust_mly GROUP BY customer_type;")

In [ ]:
def mask_query(sql):
    masked_sql = schema_link.masking_query(sql).replace('\n',' ').strip()
    cleaned_query = re.sub(r'\b(?:AS|as)\s+\w+\b', '', masked_sql)
    return cleaned_query

In [ ]:
q = """SELECT customer_type as eiei, SUM(ntx) AS TOTAL
FROM pointx_cust_mly 
GROUP BY customer_type;"""
mask_query(q)

In [ ]:
sql_df_copy = sql_df.copy()
columns = sql_df_copy.columns

for col in columns:
    print(col)
    new_col = f"MASK {col}"
    sql_df_copy[new_col] = sql_df_copy[col].apply(mask_query)

In [ ]:
sql_df_copy.to_excel("results/temp_experiments.xlsx", index=False)

# Spider

In [ ]:
import pandas as pd
import os, json, sqlite3, re
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
folder_path = "../src/spider/database"

db = dict()

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    files = os.listdir(folder_path)
    for db_id in files:
        db_path = os.path.join(folder_path, db_id)
        sqlite_db = [os.path.join(db_path, sql) for sql in os.listdir(db_path) if ".sqlite" in sql]
        assert len(sqlite_db) == 1
        db[db_id] = sqlite_db[0]


In [ ]:
def get_schema(sqlite_db_path):
    connection = sqlite3.connect(sqlite_db_path)
    cursor = connection.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    full_sql = ""
    for table in tables:
        table_name = table[0]
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()

        sql = f"CREATE TABLE {table_name} ("
        for column in columns:
            column_name = column[1]
            column_datatype = column[2].lower()
            sql += f"{column_name} {column_datatype}, "
        sql = sql[:-2] + ");"
        full_sql += sql
    
    cursor.close()
    connection.close()
    return full_sql

def create_prompt(question, schema):
    full_prompt = ""
    full_prompt += f"{str(schema)}\n\n"
    full_prompt += "-- Using valid SQLite, answer the following questions for the tables provided above.\n\n"
    full_prompt += f"--{question}\n\nSELECT"
    return full_prompt

def extract_table_and_columns(sql_statements):
    # Regular expression pattern to match the table name and column names
    pattern = r'CREATE\s+TABLE\s+(\w+)\s*\((.+?)\);?'

    table_column_pairs = {}
    matches = re.finditer(pattern, sql_statements, re.IGNORECASE)
    for match in matches:
        table_name = match.group(1)
        columns = [col.strip().split()[0] for col in match.group(2).split(',')]
        table_column_pairs[table_name] = columns

    return table_column_pairs

def table_column_of_create_table(query):
    lines = query.splitlines()
    columns = []
    table_names = []

    # Look for "CREATE TABLE" and start capturing columns
    capture = False
    for line in lines:
        if "CREATE TABLE" in line:
            capture = True
            table_names.append(line.split()[-2])
        elif line.strip().endswith(')') or line.strip().endswith(');'):
            capture = False
        elif capture:
            column_name = line.strip().split()[0]
            if column_name in ["CONSTRAINT", "PRIMARY"]: continue
            columns.append(column_name)
    return table_names, columns

def query_db(sql_query, db_name):
    try:
        conn = sqlite3.connect(f'../src/spider/database/{db_name}/{db_name}.sqlite')
        cursor = conn.cursor()
    except:
        return "CANNOT CONNECT DATABASE"
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
    except:
        return "CANNOT FETCHING DATA"
    conn.close()
    return results


In [ ]:
def spider_masking_query(sql_query:str, tab_columns:list, condition_value_mask:bool=True) -> str:

        tab_columns_lower = [t.lower() for t in tab_columns]
        if '*' in sql_query: sql_query = sql_query.replace('*', "[MASK]")
        query_split = re.split(r'(?<=[() .,;])|(?=[() .,;])', sql_query)
        mask_next = False

        for i in range(len(query_split)):
            token = query_split[i].lower()
            # prepare mask condition value
            if token.lower() == 'where': mask_next = True
            if condition_value_mask and mask_next and (token in {'=', '>', '<', '>=', '<=', '<>', '!='} and i + 1 < len(query_split)):
                step_mask_next = 1
                # find the condition value
                while query_split[i + step_mask_next] == ' ': step_mask_next += 1
                query_split[i + step_mask_next] = "[MASK]"
            
            if token in tab_columns_lower:
                query_split[i] = "[MASK]"

        return "".join(query_split)

In [ ]:
load_dotenv('../.env')
tokenizer = AutoTokenizer.from_pretrained("../models/nsql-350M")
model = AutoModelForCausalLM.from_pretrained("../models/nsql-350M")

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
DEEPSEEK_API_KEY = os.environ.get('DEEPSEEK_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

system_content_schemaprovide = """You are a helpful assistant for generate SQL query from user-specified questions and schema.
User provides you with a question.
Please return only a sql string query results.
Do not return any other format the user has provided to you.
This is example of output format which user expect from you
query : 'SELECT...'
"""

system_content_fillmask = """You are a helpful assistant for generate SQL query from user-specified questions and schema. 
User has some SQL where the [MASK] columns and condition values are syntaxed and User wants you to respond to output that populates the [MASK] column of the SQL input followed by the question and schema description (name - description).
If you don't know which column to fill in Do not include columns that you have created yourself. And only columns defined from the schema must be used. 
Do not use columns from other tables or schema. must also be used from the same table defined in the input.
If you must enter conditional values Please decide the format or value based on the sample values of that column.
If that column has many too long category value please decide base on column description.
please return only the answer of sql string query result!!! ('SELECT...')
"""

zero_shot_prompt = """For example:
table :     cat - this table contain cat information 
columns :    id - number for identify cat
            name - name of cat 
            age - age of cat 
            birth_date - pet birthday in format 'YYYY-MM-DD'
            gender - gender of cat (male, female)

question : 'Show me number of cat for each gender which born before March 23, 2011.'
query : 'SELECT gender, COUNT(*) FROM cat WHERE birth_date < '2011-03-23' GROUP BY gender;'

"""

zero_shot_prompt_mask = """For example:
table :     cat - this table contain cat information 
columns :    id - number for identify cat
            name - name of cat 
            age - age of cat 
            birth_date - pet birthday in format 'YYYY-MM-DD'
            gender - gender of cat (male, female)

question : Show me number of cat for each gender which born before March 23, 2011.
input : SELECT [MASK], COUNT([MASK]) FROM cat WHERE [MASK] < [MASK] GROUP BY [MASK] ;
query : SELECT gender, COUNT(*) FROM cat WHERE birth_date < '2011-03-23' GROUP BY gender;

"""

In [ ]:
def generate_nsql_sql(prompt):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids
        generated_ids = model.generate(input_ids, max_length=1000)
        sql = tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('\n')[-1]
    return sql

In [ ]:
with open("../src/src_dev/spider/mockup_schema_description.json") as f:
    spider_description = json.load(f)
    exists_tables = [tab['table'] for tab in spider_description]

with open("../src/src_dev/spider/table_database_map.json") as f:
    table_map_db = json.load(f)

In [ ]:
spider_df = pd.read_csv('../src/NSText2SQL/train_spider.csv')
spider_df.head()

In [ ]:
predict_data = {
    "Question" : [],
    "Desc DeepSeek" : [],
    "Desc GPT3.5" : [],
    "Desc GPT4" : [],
    "Desc Gemini" : [],
    "ChatQ NSQL" : [],
    "ChatQ DeepSeek" : [],
    "ChatQ GPT3.5" : [],
    "ChatQ GPT4" : [],
    "ChatQ Gemini" : []
}

temp_result_file = "results/temp_spider_experiments.xlsx"

if os.path.exists(temp_result_file):
    print("File exist")
    _df = pd.read_excel(temp_result_file)
    i = _df.shape[0]
    for key in predict_data:
        if key in _df.columns:
            predict_data[key] = _df[key].tolist()

for i in range(spider_df.shape[0]):
    tabs, cols = table_column_of_create_table(spider_df.iloc[i,1])
    if len(set(tabs).intersection(set(exists_tables))) == len(tabs) :

        question = spider_df.iloc[i,0]
        if question in predict_data['Question']: continue
        nsql_prompt = create_prompt(question, spider_df.iloc[i,1])
        actual_sql = spider_df.iloc[i,2]
        
        schema_desc_prompt = ""
        for use_tabl in tabs:
            used_schema = spider_description[exists_tables.index(use_tabl)]
            schema_desc_prompt += f"{used_schema['table']} - {used_schema['description']}\n"
            for col_name, col_desc in used_schema['columns'].items():
                schema_desc_prompt += f"\t{col_name} - {col_desc}\n"

        schema_desc_prompt += f"question: {question}\n"

        schema_provide_prompt = zero_shot_prompt + schema_desc_prompt + "query: "

        schema_gemini_results = LLM_gensql(schema_provide_prompt, system_content_schemaprovide, 'gemini-pro')
        schema_gpt3_5_results = LLM_gensql(schema_provide_prompt, system_content_schemaprovide, 'gpt-3.5-turbo')
        schema_gpt4_results = LLM_gensql(schema_provide_prompt, system_content_schemaprovide, 'gpt-4-0125-preview')
        schema_deepseek_result = LLM_gensql(schema_provide_prompt, system_content_schemaprovide, 'deepseek-coder')
        
        nsql_result = generate_nsql_sql(nsql_prompt)
        masked_nsql_result = spider_masking_query(nsql_result, cols)
        
        chatq_prompt = zero_shot_prompt_mask + schema_desc_prompt + f"input : {masked_nsql_result}\nquery: "

        chatq_gemini_results = LLM_gensql(chatq_prompt, system_content_fillmask, 'gemini-pro')
        chatq_gpt3_5_results = LLM_gensql(chatq_prompt, system_content_fillmask, 'gpt-3.5-turbo')
        chatq_gpt4_results = LLM_gensql(chatq_prompt, system_content_fillmask, 'gpt-4-0125-preview')
        chatq_deepseek_result = LLM_gensql(chatq_prompt, system_content_fillmask, 'deepseek-coder')

        print("Generating SQL...")
        schema_gemini_results, schema_deepseek_result, schema_gpt3_5_results, schema_gpt4_results = await asyncio.gather(schema_gemini_results, schema_deepseek_result, schema_gpt3_5_results, schema_gpt4_results)
        chatq_gemini_results, chatq_gpt3_5_results, chatq_gpt4_results, chatq_deepseek_result = await asyncio.gather(chatq_gemini_results, chatq_gpt3_5_results, chatq_gpt4_results, chatq_deepseek_result)

        print(question)
        print(schema_gemini_results, schema_deepseek_result, schema_gpt3_5_results, schema_gpt4_results, chatq_gemini_results, chatq_gpt3_5_results, chatq_gpt4_results, chatq_deepseek_result, sep='\n')

        predict_data['Question'].append(question)
        predict_data['Desc DeepSeek'].append(schema_deepseek_result)
        predict_data['Desc GPT3.5'].append(schema_gpt3_5_results)
        predict_data['Desc GPT4'].append(schema_gpt4_results)
        predict_data['Desc Gemini'].append(schema_gemini_results)
        predict_data['ChatQ NSQL'].append(nsql_result)
        predict_data['ChatQ DeepSeek'].append(chatq_deepseek_result)
        predict_data['ChatQ GPT3.5'].append(chatq_gpt3_5_results)
        predict_data['ChatQ GPT4'].append(chatq_gpt4_results)
        predict_data['ChatQ Gemini'].append(chatq_gemini_results)

        save_df = pd.DataFrame(predict_data)
        save_df.to_excel(temp_result_file, index=False)
        print("SAVE TEMP COMPLETE", i)

save_df = pd.DataFrame(predict_data)
save_df.to_excel("results/model_spider_experiments.xlsx", index=False)

In [ ]:
len(chatq_prompt)

In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv('../.env')
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)
response

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
for i in 